In [1]:
with open("medical_faq.txt") as f:
    med_faq = f.read()

In [20]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)
docs = text_splitter.create_documents([med_faq])

Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 185, which is longer than the specified 100
Created a chunk of size 159, which is longer than the specified 100
Created a chunk of size 155, which is longer than the specified 100
Created a chunk of size 153, which is longer than the specified 100
Created a chunk of size 160, which is longer than the specified 100
Created a chunk of size 157, which is longer than the specified 100
Created a chunk of size 142, which is longer than the specified 100
Created a chunk of size 155, which is longer than the specified 100
Created a chunk of size 155, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 109, which is longer tha

In [21]:
docs[0]

Document(page_content='[患者问题]：如何预防感冒？\n   [医生回答]：勤洗手，避免接触感冒患者，多喝水，保持良好的生活习惯，加强锻炼，提高免疫力。')

In [22]:
len(docs)

211

In [23]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [25]:
query = "996怕猝死怎么办"
answer_list = db.similarity_search(query)
for ans in answer_list:
    print(ans.page_content + "\n")

[患者问题]：程序员如何规避996工作制下的猝死风险？
    [医生回答]：长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。

[患者问题]：熬夜工作如何降低猝死风险？
    [医生回答]：尽量避免长期熬夜，保证充足的睡眠，定期进行体检，特别是心脏健康检查，保持健康的生活方式，如不吸烟、适量运动。

[患者问题]：抑郁症患者如何应对自杀冲动？
    [医生回答]：抑郁症患者应寻求专业心理治疗和药物治疗。建立支持系统，与家人、朋友和医生保持密切联系。学习应对自杀冲动的策略，如寻求帮助、参加心理支持和小组活动。

[患者问题]：中年男性如何应对工作压力导致的健康问题？
    [医生回答]：中年男性应学会合理安排工作和休息，进行适量的运动，保持良好的饮食习惯。如有焦虑、抑郁等症状，可寻求心理咨询或心理治疗。定期进行体检，及时发现和治疗潜在健康问题。



In [26]:
db.save_local("med_faq")

In [28]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x119ceff10>, search_kwargs={'k': 3})

In [29]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[患者问题]：程序员如何规避996工作制下的猝死风险？
    [医生回答]：长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。

[患者问题]：熬夜工作如何降低猝死风险？
    [医生回答]：尽量避免长期熬夜，保证充足的睡眠，定期进行体检，特别是心脏健康检查，保持健康的生活方式，如不吸烟、适量运动。

[患者问题]：抑郁症患者如何应对自杀冲动？
    [医生回答]：抑郁症患者应寻求专业心理治疗和药物治疗。建立支持系统，与家人、朋友和医生保持密切联系。学习应对自杀冲动的策略，如寻求帮助、参加心理支持和小组活动。



In [30]:
docs = topK_retriever.get_relevant_documents("最近有点emo怎么办")
for doc in docs:
    print(doc.page_content + "\n")

[患者问题]：双相情感障碍患者如何管理情绪波动？
    [医生回答]：双相情感障碍患者应遵循医嘱，按时服用情绪稳定剂和抗抑郁药物。学习情绪调节技巧，如冥想、放松和情绪日记。保持健康的生活方式，避免刺激性物质和过度劳累。

[患者问题]：抑郁症患者如何应对自杀冲动？
    [医生回答]：抑郁症患者应寻求专业心理治疗和药物治疗。建立支持系统，与家人、朋友和医生保持密切联系。学习应对自杀冲动的策略，如寻求帮助、参加心理支持和小组活动。

[患者问题]：强迫症患者如何应对强迫思维和行为？
    [医生回答]：强迫症患者应遵循医嘱，按时服用抗强迫药物和抗抑郁药物。学习认知行为疗法，特别是暴露和反应预防（ERP）技术。参与支持小组，与他人分享经验和策略。



In [31]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [32]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

/Users/wangkang/anaconda3/lib/python3.11/site-packages/langchain_core/vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [36]:
retriever_75 = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.75}
)
docs = retriever_75.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[患者问题]：程序员如何规避996工作制下的猝死风险？
    [医生回答]：长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。



In [38]:
docs = retriever_75.get_relevant_documents(query)
docs[0].page_content

'[患者问题]：程序员如何规避996工作制下的猝死风险？\n    [医生回答]：长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。'

In [42]:
docs[0].page_content.split("[医生回答]：")

['[患者问题]：程序员如何规避996工作制下的猝死风险？\n    ',
 '长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。']

In [43]:
ans = docs[0].page_content.split("[医生回答]：")[-1]
ans

'长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。'

In [45]:
from typing import List

def med(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[医生回答]：")[-1] for doc in docs]

    return ans_list

In [46]:
med("女朋友痛经了怎么办")

['痛经可能由多种原因引起，包括子宫内膜异位症等。可以使用非处方止痛药，如布洛芬，来缓解疼痛。同时，保持良好的生活习惯，如规律饮食和适量运动，有助于减轻痛经症状。']

In [49]:
med("没力气写代码了怎么办", score_threshold=0.7)

['程序员应学会合理安排工作和休息，进行适量的运动，保持良好的饮食习惯。寻求心理支持和咨询，学习压力管理技巧。',
 '程序员应定期进行休息和放松，如短暂的休息、伸展运动或短暂的散步。学习时间管理和任务优先级，保持良好的工作生活平衡。',
 '程序员应学会时间管理和任务优先级，保持良好的工作生活平衡。进行适量的运动，保持良好的饮食习惯，并寻求心理咨询或治疗。',
 '程序员应积极参与团队活动和社交活动，培养良好的沟通技巧，寻求心理支持和咨询。同时，保持健康的生活方式，如适量运动和均衡饮食。']

In [50]:
med("牙疼怎么办")

['牙痛可能是由多种原因引起的，如牙髓炎、牙周炎等。首先，应避免食用刺激性食物，保持口腔卫生。如果牙痛持续或加剧，应及时就医。医生可能会进行局部麻醉，然后进行根管治疗或牙周治疗。',
 '保持良好的口腔卫生习惯，定期刷牙和使用牙线，避免吸烟，定期进行口腔检查和专业清洁。',
 '牙齿敏感可能是由于牙釉质磨损或牙龈退缩引起的。使用软毛牙刷，避免用力刷牙。定期进行牙科检查，接受专业的牙周治疗。在医生指导下，使用抗敏感牙膏或进行脱敏治疗。',
 '牙齿咬合问题可能导致牙齿磨损、疼痛或颞下颌关节紊乱。以下是一些处理建议：\n    - 进行全面的口腔检查，评估咬合关系。\n    - 在必要时，进行正畸治疗或咬合板治疗，以改善咬合关系。\n    - 避免使用牙齿作为工具，如用牙齿开瓶盖或撕开包装。\n    - 定期进行牙科检查，评估咬合状况的变化。']

In [52]:
med("胃疼怎么办")

['保持规律的饮食习惯，避免暴饮暴食，少吃油腻和辛辣食物，多喝水，必要时进行药物治疗。',
 '保持规律的饮食习惯，避免过量进食和辛辣油腻食物，适量运动，必要时使用抗酸药或消化酶补充剂。',
 '中医调理胃痛可以通过服用具有健脾和胃作用的中药，如白术、茯苓等，以及调整饮食和生活习惯，如避免辛辣刺激性食物，保持饮食规律。',
 '中医治疗慢性胃炎可以通过服用具有健脾和胃作用的中药，如白术、茯苓等，以及调整饮食和生活习惯，如避免辛辣刺激性食物，保持饮食规律。']

In [53]:
med("如何降低猝死风险")

['尽量避免长期熬夜，保证充足的睡眠，定期进行体检，特别是心脏健康检查，保持健康的生活方式，如不吸烟、适量运动。',
 '长期过劳工作和高压环境可能增加心脏疾病和猝死的风险。为规避这些风险，程序员应定期进行体检，特别是心脏健康检查，确保及时发现和处理潜在的健康问题。同时，应保持健康的生活方式，如适量运动、均衡饮食、充足睡眠和避免吸烟。此外，学习压力管理和放松技巧，如冥想、深呼吸和瑜伽，也有助于减轻工作压力。']

In [55]:
med("太胖了怎么办", score_threshold=0.75)

['鼓励健康的饮食习惯，限制高糖和高脂肪食物，增加户外活动和体育锻炼，限制屏幕时间。',
 '保持健康的饮食习惯，如多吃蔬菜水果和全谷物，少吃高糖和高脂肪食物，定期进行锻炼，如散步、跑步或健身。',
 '定期站立和走动，进行办公室锻炼，如伸展和椅子运动，控制零食摄入，保持健康的饮食习惯，合理安排午餐和休息时间。',
 '中医调理肥胖可以通过服用具有利湿化痰和健脾消脂作用的中药，如荷叶、山楂等，以及调整饮食和生活习惯，如控制饮食热量，增加运动量。']

In [56]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("med_faq", OpenAIEmbeddings())

/Users/wangkang/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [59]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.75}))

In [60]:
qa_chain({"query": "程序员怎么养生呀"})

/Users/wangkang/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '程序员怎么养生呀',
 'result': '程序员养生的方法有很多，以下是一些建议：\n\n1. 合理安排工作和休息时间：避免长时间连续工作，每隔一段时间就要休息一下，可以做些伸展运动或眼保健操，放松身心。\n\n2. 均衡饮食：保持健康的饮食习惯，多吃蔬菜水果和全谷物，少吃高糖和高脂肪食物。避免过度依赖咖啡因和能量饮料。\n\n3. 定期运动：进行适量的运动，如散步、跑步、健身等，有助于提高身体素质和心理健康。\n\n4. 管理压力：学习压力管理技巧，如寻求心理支持和咨询，学习放松技巧，如冥想、深呼吸等。\n\n5. 维护良好的工作生活平衡：不要让工作占据全部时间和精力，合理安排工作和生活的时间，培养自己的兴趣爱好，与家人和朋友保持良好的社交关系。\n\n请注意，以上建议仅供参考，具体养生方法还需根据个人情况来调整和选择。如果有特殊的健康问题，建议咨询医生或专业人士的意见。'}

In [61]:
qa_chain.combine_documents_chain.verbose = True
qa_chain({"query": "程序员怎么养生呀"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '程序员怎么养生呀',
 'result': '程序员养生的方法包括以下几点：\n\n1. 合理安排工作和休息：确保每天有足够的休息时间，避免长时间连续工作，可以定时休息或进行短暂的放松活动。\n\n2. 健康饮食：保持良好的饮食习惯，多吃蔬菜水果和全谷物，少吃高糖和高脂肪食物。避免过度依赖咖啡因和能量饮料。\n\n3. 适量运动：定期进行身体锻炼，如散步、跑步、健身等，有助于缓解身体的紧张和疲劳感。\n\n4. 心理支持和咨询：寻求心理支持和咨询，学习压力管理技巧，可以通过与同事、朋友或专业心理咨询师交流来减轻心理压力。\n\n5. 维护工作生活平衡：学会时间管理和任务优先级，合理安排工作和生活，确保有足够的休闲和娱乐时间。\n\n请注意，以上建议仅供参考，具体养生方法还需根据个人情况和健康状况进行调整。如有需要，建议咨询专业医生或健康专家。'}

In [62]:
qa_chain({"query": "感冒了嗓子疼怎么办"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '感冒了嗓子疼怎么办',
 'result': '可以尝试以下方法缓解感冒引起的喉咙疼痛：\n1. 多喝温水：保持身体水分充足，有助于稀释病毒和减轻喉咙疼痛。\n2. 喉咙喷雾：使用含有局部麻醉剂或舒缓成分的喉咙喷雾，可以暂时缓解喉咙疼痛。\n3. 盐水漱口：用温盐水漱口，可以舒缓喉咙疼痛和减少病毒繁殖。\n4. 喉咙糖或喉咙片：含有舒缓成分的喉咙糖或喉咙片，可以缓解喉咙疼痛。\n5. 休息好：充足的休息有助于身体恢复，也可以减轻喉咙疼痛的不适感。\n如果喉咙疼痛严重或持续时间较长，建议咨询医生进行进一步的评估和治疗。'}